# Preprocessing of the trade network

In [16]:
def net_symmetrisation(wtn_file, exclude_countries):
    DG=nx.DiGraph()
    
    Reporter_pos=1
    Partner_pos=3
    Flow_code_pos=2
    Value_pos=9
    
    dic_trade_flows={}
    hfile=open(wtn_file,'r')
    
    header=hfile.readline()
    lines=hfile.readlines()
    for l in lines:
        l_split=l.split(',')
        #the following is to prevent parsing lines without data
        if len(l_split)<2: continue 
        reporter=int(l_split[Reporter_pos])
        partner=int(l_split[Partner_pos])
        flow_code=int(l_split[Flow_code_pos])
        value=int(l_split[Value_pos])
        
        if ( (reporter in exclude_countries) or \
            (partner in exclude_countries) or (reporter==partner) ): 
            continue
        
        if flow_code==1 and value>0.0:
            #1=Import, 2=Export
            if (partner,reporter,2) in dic_trade_flows: 
                DG[partner][reporter]['weight']= \
                (DG[partner][reporter]['weight']+value)/2.0
            else:
                DG.add_edge(partner, reporter, weight=value)
                dic_trade_flows[(partner,reporter,1)]= \
                value #this is to mark the exixtence of the link
        
        elif flow_code==2 and value>0.0:
            #1=Import, 2=Export
            if (reporter,partner,1) in dic_trade_flows: 
                DG[reporter][partner]['weight']= \
                (DG[reporter][partner]['weight']+value)/2.0
            else:
                DG.add_edge(reporter, partner, weight=value)
                #this is to mark the exixtence of the link
                dic_trade_flows[(reporter,partner,2)]=value 
        else:
            print("trade flow not present\n")
        
    hfile.close()
    
    return DG

In [17]:
import networkx as nx
#countries to be excluded
exclude_countries=[472,899,471,129,221,97,697,492,838,473,536, 637,290,527,577,490,568,636,839,879,0]
DG = net_symmetrisation("../Data/Comtrade/comtrade_trade_data_total_2003.csv", exclude_countries)

In [18]:
#import the country name
import csv

country_dict = {}

with open("../Data/Comtrade/Country Code and Name ISO2 ISO3_annotated.csv", "r") as csvfile:
    reader = csv.reader(csvfile)
    next(reader)
    for row in reader:
        country_dict[int(row[0])] =  row[1]

In [19]:
for node in DG.nodes():
    DG.node[node]["name"] = country_dict[node]

In [20]:
nx.write_gpickle(DG, "../Data/Comtrade/comtrade.gpickle")

In [21]:
nx.write_gexf(DG, "../Data/Comtrade/comtrade.gexf")

In [22]:
nx.write_pajek(DG, "../Data/Comtrade/comtrade.net")

In [23]:
nx.write_gml(DG, "../Data/Comtrade/comtrade.gml")